In [ ]:
# здесь реализован 3-ий шаг задания. Я сразу сделала модели с кросс-валидацией, но модель со случайным лесом продемонстрировала 
# практически идеальные результаты, что насторожило, решила сравнить эти модели без и с-кросс-валидацией

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [2]:
# Загрузка данных
data = pd.read_csv('telcom_customer_churn.csv')

In [3]:
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

In [4]:
# Заполнение NaN в строковых столбцах модой
for column in data.select_dtypes(include=['object']).columns:
    mode_value = data[column].mode()[0]  # Вычисляем моду
    data[column] = data[column].fillna(mode_value)  # Заполняем NaN модой

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 100 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   rev_Mean          100000 non-null  float64
 1   mou_Mean          100000 non-null  float64
 2   totmrc_Mean       100000 non-null  float64
 3   da_Mean           100000 non-null  float64
 4   ovrmou_Mean       100000 non-null  float64
 5   ovrrev_Mean       100000 non-null  float64
 6   vceovr_Mean       100000 non-null  float64
 7   datovr_Mean       100000 non-null  float64
 8   roam_Mean         100000 non-null  float64
 9   change_mou        100000 non-null  float64
 10  change_rev        100000 non-null  float64
 11  drop_vce_Mean     100000 non-null  float64
 12  drop_dat_Mean     100000 non-null  float64
 13  blck_vce_Mean     100000 non-null  float64
 14  blck_dat_Mean     100000 non-null  float64
 15  unan_vce_Mean     100000 non-null  float64
 16  unan_dat_Mean     10

In [5]:
# 'churn' - целевая переменная
X = data.drop('churn', axis=1)
y = data['churn']

In [6]:
# Предварительная обработка данных
num_features = X.select_dtypes(include=['int64', 'float64']).columns  # Числовые признаки
cat_features = X.select_dtypes(include=['object']).columns  # Категориальные признаки

# Создание трансформеров
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение трансформеров
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# Функция для оценки модели без кросс-валидации

def evaluate_model_without_cv(pipeline, X_train, y_train, X_test, y_test):
    # Обучение модели
    pipeline.fit(X_train, y_train)

    # Предсказания
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

    # Оценка метрик
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'ROC AUC': roc_auc_score(y_test, y_pred_proba)
    }

    return metrics

In [10]:
# Логистическая регрессия
logistic_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', LogisticRegression(max_iter=500))])
logistic_metrics = evaluate_model_without_cv(logistic_pipeline, X_train, y_train, X_test, y_test)

In [11]:
# Случайный лес
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(random_state=42))])
rf_metrics = evaluate_model_without_cv(rf_pipeline, X_train, y_train, X_test, y_test)

In [12]:
# Градиентный бустинг
gb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', GradientBoostingClassifier(random_state=42))])
gb_metrics = evaluate_model_without_cv(gb_pipeline, X_train, y_train, X_test, y_test)

In [13]:
# Собираем результаты
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    'Metrics': [logistic_metrics, rf_metrics, gb_metrics]
})
results.to_string()


"                 Model                                                                                                                                                         Metrics\n0  Logistic Regression              {'Accuracy': 0.5901, 'Precision': 0.5896644295302014, 'Recall': 0.5869463558019907, 'F1 Score': 0.5883022531721852, 'ROC AUC': 0.6273950441183884}\n1        Random Forest  {'Accuracy': 0.6181333333333333, 'Precision': 0.6199713134348747, 'Recall': 0.6063865321664774, 'F1 Score': 0.6131036811887876, 'ROC AUC': 0.6656620208942314}\n2    Gradient Boosting  {'Accuracy': 0.6280666666666667, 'Precision': 0.6220300992635287, 'Recall': 0.6488743403032935, 'F1 Score': 0.6351687156683233, 'ROC AUC': 0.6810971245970522}"

In [24]:
# Преобразуем метрики в отдельные колонки
metrics_df = results['Metrics'].apply(pd.Series)
results = pd.concat([results[['Model']], metrics_df], axis=1)

# Выбор лучшей модели по метрике (например, по 'ROC AUC')
best_model = results.loc[results['ROC AUC'].idxmax()]

# Вывод лучшей модели
print("Лучшая модель:")
print(best_model)

Лучшая модель:
Model        Gradient Boosting
Accuracy              0.628067
Precision              0.62203
Recall                0.648874
F1 Score              0.635169
ROC AUC               0.681097
Name: 2, dtype: object


In [16]:
# пробовало модель с опорными векторами,больше суток считал (и это без кросс-валидации), нервы сдали, попробовала стекинг
# Создание базовых моделей
base_models = [
    ('logistic', LogisticRegression(max_iter=500)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
]

In [17]:
# Создание стекинг классификатора
stacking_classifier = StackingClassifier(
    estimators=base_models, 
    final_estimator=LogisticRegression(),
    cv=5
)

In [18]:
# Создание полного конвейера
stacking_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', stacking_classifier)])


In [19]:
# Обучение модели
stacking_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['rev_Mean', 'mou_Mean', 'totmrc_Mean', 'da_Mean', 'ovrmou_Mean',
       'ovrrev_Mean', 'vceovr_Mean', 'datovr_Mean', 'roam_Mean', 'change_mou',
       'change_rev', 'drop_vce_Mean', 'drop_dat_Mean', 'blck_vce_Mean',
       'blck_dat_Mean', 'unan_vce_Mean', 'unan_...
       'marital', 'infobase', 'HHstatin', 'dwllsize', 'ethnic', 'kid0_2',
       'kid3_5', 'kid6_10', 'kid11_15', 'kid16_17', 'creditcd'],
      dtype='object'))])),
                ('classifier',
                 StackingClassifier(cv=5,
                                    estimators=[('logistic',
                                                 LogisticRegression(max_iter=500)),
                                                ('rf',
                                                 RandomForestClassifier(random_state=42)),
                                                ('gb',
                                                 GradientBoostingClassifier(random_state=42))],
                                    final_estimator=LogisticRegression()))])

In [20]:
# Предсказание
y_pred = stacking_pipeline.predict(X_test)
y_pred_proba = stacking_pipeline.predict_proba(X_test)[:, 1]

In [22]:
# Оценка метрик
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)


In [23]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

Accuracy: 0.6319
Precision: 0.6256
Recall: 0.6529
F1 Score: 0.6390
ROC AUC: 0.6850


In [ ]:
# таким образом, стекинг обошел по показателям бустинг

In [25]:
# теперь проведем кросс-валидацию
# Функция для оценки модели c кросс-валидацией
def evaluate_model(pipeline, X, y):
    # Кросс-валидация
    cv_scores = cross_val_score(pipeline, X, y, cv=5)  # Используем 5 фолдов
    mean_cv_score = cv_scores.mean()

    # Обучение модели на всех данных
    pipeline.fit(X, y)

    # Предсказания
    y_pred = pipeline.predict(X)
    y_pred_proba = pipeline.predict_proba(X)[:, 1]

    # Оценка метрик
    metrics = {
        'CV Accuracy': mean_cv_score,
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred),
        'Recall': recall_score(y, y_pred),
        'F1 Score': f1_score(y, y_pred),
        'ROC AUC': roc_auc_score(y, y_pred_proba)
    }

    return metrics

In [26]:
# Обучение различных моделей

# Логистическая регрессия
logistic_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', LogisticRegression(max_iter=500))])
logistic_metrics = evaluate_model(logistic_pipeline, X_train, y_train)

In [27]:
# Случайный лес
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(random_state=42))])
rf_metrics = evaluate_model(rf_pipeline, X_train, y_train)


In [21]:
# Градиентный бустинг
gb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', GradientBoostingClassifier(random_state=42))])
gb_metrics = evaluate_model(gb_pipeline, X_train, y_train)


In [28]:
# Стекинг
from sklearn.ensemble import StackingClassifier

# Определяем базовые модели для стекинга
base_estimators = [
    ('logistic', LogisticRegression(max_iter=500)),
    ('random_forest', RandomForestClassifier(random_state=42)),
    ('gradient_boosting', GradientBoostingClassifier(random_state=42))
]

# Создаем стекинг-классификатор
#stacking_pipeline = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression(max_iter=500))
stacking_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Применение предварительной обработки
    ('stacking', StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression(max_iter=500)))
])

In [29]:
# Оцениваем модель стекинга
stacking_metrics = evaluate_model(stacking_pipeline, X_train, y_train)

In [31]:
# Собираем результаты
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'Gradient Boosting', 'Stacking'],
    'Metrics': [logistic_metrics, rf_metrics, gb_metrics, stacking_metrics]
})

# Вывод результатов
print(results.to_string())

                 Model                                                                                                                                                                                           Metrics
0  Logistic Regression             {'CV Accuracy': 0.5941, 'Accuracy': 0.5990714285714286, 'Precision': 0.5941447937698299, 'Recall': 0.5954672910704478, 'F1 Score': 0.5948053073069316, 'ROC AUC': 0.6357599853108626}
1        Random Forest                                                                            {'CV Accuracy': 0.6165285714285714, 'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1 Score': 1.0, 'ROC AUC': 1.0}
2    Gradient Boosting                                    {'Accuracy': 0.6280666666666667, 'Precision': 0.6220300992635287, 'Recall': 0.6488743403032935, 'F1 Score': 0.6351687156683233, 'ROC AUC': 0.6810971245970522}
3             Stacking  {'CV Accuracy': 0.6314571428571428, 'Accuracy': 0.9328571428571428, 'Precision': 0.9438719448816559, 'Recall

In [32]:
# Преобразуем метрики в отдельные колонки
metrics_df = results['Metrics'].apply(pd.Series)
results = pd.concat([results[['Model']], metrics_df], axis=1)

# Выбор лучшей модели по метрике (например, по 'ROC AUC')
best_model = results.loc[results['ROC AUC'].idxmax()]

# Вывод лучшей модели
print("Лучшая модель:")
print(best_model)

Лучшая модель:
Model          Random Forest
CV Accuracy         0.616529
Accuracy                 1.0
Precision                1.0
Recall                   1.0
F1 Score                 1.0
ROC AUC                  1.0
Name: 1, dtype: object


In [33]:
# Выбор лучшей модели по метрике (например, по 'ROC AUC')
best_model = results.loc[results['CV Accuracy'].idxmax()]

# Вывод лучшей модели
print("Лучшая модель:")
print(best_model)

Лучшая модель:
Model          Stacking
CV Accuracy    0.631457
Accuracy       0.932857
Precision      0.943872
Recall          0.91877
F1 Score       0.931152
ROC AUC        0.983222
Name: 3, dtype: object
